<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#データ" data-toc-modified-id="データ-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>データ</a></span><ul class="toc-item"><li><span><a href="#モデル" data-toc-modified-id="モデル-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>モデル</a></span></li></ul></li></ul></div>

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import trange

In [26]:
import sys, os

## データ

In [27]:
transform = transforms.Compose(
    [
        transforms.ToTensor(), # テンソル化
        transforms.Normalize(  # 正規化
            (0.5, 0.5, 0.5), # 平均
            (0.5, 0.5, 0.5)  # 標準偏差
        )
    ]
)

trainset=CIFAR10(root='../data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

Files already downloaded and verified


### モデル

In [28]:
# https://pystyle.info/pytorch-gan/

In [29]:
def get_device(gpu_id=-1):
    if gpu_id >= 0 and torch.cuda.is_available():
        return torch.device("cuda", gpu_id)
    else:
        return torch.device("cpu")


device = get_device(gpu_id=0)

In [30]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.main = nn.Sequential(
            # fc1
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            # fc2
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            # fc3
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            # fc4
            nn.Linear(128, 1),
            nn.Sigmoid(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.main(x)

In [31]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.main = nn.Sequential(
            # fc1
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            # fc2
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2, inplace=True),
            # fc3
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            # fc4
            nn.Linear(512, output_dim),
            nn.Tanh(),
        )

    def forward(self, x):
        return self.main(x)

In [32]:
latent_dim = 100  # ノイズの次元数
data_dim = 28 * 28  # データの次元数

# 学習過程で Generator が生成する画像を可視化するためのノイズ z
fixed_z = torch.randn(100, latent_dim, device=device)

# ラベル
real_label = 1
fake_label = 0

# Generator を作成する。
G = Generator(latent_dim, data_dim).to(device)
# Discriminator を作成する。
D = Discriminator(data_dim).to(device)

In [33]:
# 損失関数を作成する。
criterion = nn.BCELoss()

# オプティマイザを作成する。
lr = 0.0002
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)

In [36]:
def D_train(x):
    D.zero_grad()
    # (N, H, W) -> (N, H * W) に形状を変換する。
    x = x.flatten(start_dim=1)

    # 損失関数を計算する。
    # 本物のデータが入力の場合の Discriminator の損失関数を計算する。
    y_pred = D(x)
    y_real = torch.full_like(y_pred, real_label)
    loss_real = criterion(y_pred, y_real)

    # 偽物のデータが入力の場合の Discriminator の損失関数を計算する。
    z = torch.randn(x.size(0), latent_dim, device=device)
    y_pred = D(G(z))
    y_fake = torch.full_like(y_pred, fake_label)
    loss_fake = criterion(y_pred, y_fake)

    loss = loss_real + loss_fake

    # 逆伝搬する。
    loss.backward()

    # パラメータを更新する。
    D_optimizer.step()

    return float(loss)

In [37]:
def G_train(x):
    G.zero_grad()

    # 損失関数を計算する。
    z = torch.randn(x.size(0), latent_dim, device=device)
    y_pred = D(G(z))
    y = torch.full_like(y_pred, real_label)
    loss = criterion(y_pred, y)

    # 逆伝搬する。
    loss.backward()

    # パラメータを更新する。
    G_optimizer.step()

    return float(loss)

In [38]:
def generate_img(G, fixed_z):
    with torch.no_grad():
        # 画像を生成する。
        x = G(fixed_z)

    # (N, C * H * W) -> (N, C, H, W) に形状を変換する。
    x = x.view(-1, 1, 28, 28).cpu()
    # 画像を格子状に並べる。
    img = torchvision.utils.make_grid(x, nrow=10, normalize=True, pad_value=1)
    # テンソルを PIL Image に変換する。
    img = transforms.functional.to_pil_image(img)

    return img

In [42]:
pip install ipywidgets widgetsnbextension pandas-profiling

  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl (261 kB)
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
  Using cached visions-0.7.4-py3-none-any.whl (102 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached Jinja2-3.1.1-py3-none-any.whl (132 kB)
  Using cached tangled_up_in_unicode-0.1.0-py3-none-any.whl (3.1 MB)
  Using cached missingno-0.5.1-py3-none-any.whl (8.7 kB)
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached multimethod-1.8-py3-none-any.whl (9.8 kB)
  Using cached attrs-21.4.0-py2.py3-none-any.whl (60 kB)
  Using cached ImageHash-4.2.1.tar.gz (812 kB)

  Using cached mistune-0.8.4-py2.py3-none-any.whl (16 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=037a18070eb5a5a25d952970108b5b5365fe9b148358d449059f21f7b5c8dcb5
  Stored in directory: c:\users\y\appdata\local\pip\cache\wheels\1d\05\04\c6d7d3b66539d9e659ac6dfe81e2d0fd

In [43]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [44]:
def train_gan(n_epoch):
    G.train()
    D.train()

    history = []
    for epoch in trange(n_epoch, desc="epoch"):

        D_losses, G_losses = [], []
        for x, _ in dataloader:
            x = x.to(device)
            D_losses.append(D_train(x))
            G_losses.append(G_train(x))

        # 途中経過を確認するために画像を生成する。
        img = generate_img(G, fixed_z)

        # 途中経過を記録する。
        info = {
            "epoch": epoch + 1,
            "D_loss": np.mean(D_losses),
            "G_loss": np.mean(G_losses),
            "img": img,
        }
        history.append(info)

    history = pd.DataFrame(history)

    return history


history = train_gan(n_epoch=50)

Exception ignored in: <function tqdm.__del__ at 0x00000209D343ADC0>
Traceback (most recent call last):
  File "C:\Users\Y\.conda\envs\py3.9.7env\lib\site-packages\tqdm\std.py", line 1147, in __del__
    self.close()
  File "C:\Users\Y\.conda\envs\py3.9.7env\lib\site-packages\tqdm\notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x00000209D343ADC0>
Traceback (most recent call last):
  File "C:\Users\Y\.conda\envs\py3.9.7env\lib\site-packages\tqdm\std.py", line 1147, in __del__
    self.close()
  File "C:\Users\Y\.conda\envs\py3.9.7env\lib\site-packages\tqdm\notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

EoF